# <center> Сегментация клиентов онлайн магазина подарков

## Постановка задачи


<center> <img src=https://salesupnow.ru/storage/app/media/pipeople.png align="right" width="300"/> </center>

Маркетинг — неотъемлемая часть любого бизнеса. Для повышения прибыли компании важно понимать своего клиента, его пожелания и предпочтения. С появлением электронной коммерции, или онлайн-продаж, стало намного проще собирать данные о клиентах, анализировать их, находить закономерности и реализовывать маркетинговые кампании.

Большинство интернет-магазинов используют инструменты веб-аналитики, чтобы отслеживать просмотры страниц, количество и поведение посетителей и коэффициент отказов. Но отчёта из Google Analytics или аналогичной системы может быть недостаточно для полного понимания того, как клиенты взаимодействуют с сайтом. Компаниям важно иметь возможность быстро и точно реагировать на перемены в поведении клиентов, создавая инструменты, которые обнаруживают эти изменения практически в режиме реального времени.

Машинное обучение помогает поисковой системе анализировать огромное количество данных о посетителях платформы, узнавать модели поведения профессиональных покупателей, определять категорию клиентов (например, лояльные/перспективные/новички/спящие/ушедшие) и выбирать правильную стратегию взаимодействия с ними.

Стоит также отметить, что компании, использующие машинное обучение на своих платформах электронной коммерции, могут постоянно повышать эффективность бизнес-процессов: настраивать товарную выборку персонально для каждого покупателя и предлагать выгодную цену в соответствии с бюджетом клиента и т. д. Эта задача относится к категории построения рекомендательных систем, речь о которых пойдёт в следующем разделе нашего курса.

> Как правило, наборы данных для электронной коммерции являются частной собственностью и, следовательно, их трудно найти среди общедоступных данных. Однако [The UCI Machine Learning Repository](http://archive.ics.uci.edu/ml/index.php)  создал набор данных, содержащий фактические транзакции за 2010 и 2011 годы. С ним нам как раз и предлагается поработать в этом кейсе. 

> В нашем распоряжении будет набор данных, который содержит все транзакции, произошедшие в период с 01/12/2010 по 09/12/2011 для базирующейся в Великобритании компании, занимающейся онлайн-розничной торговлей. Компания в основном продает уникальные подарки на все случаи жизни. Многие клиенты компании являются оптовиками.


**Бизнес-задача:** произвести сегментацию существующих клиентов, проинтерпретировать эти сегменты и определить стратегию взаимодействия с ними.

**Техническая задача для вас как для специалиста в Data Science:** построить модель кластеризации клиентов на основе их покупательской способности, частоты заказов и срока давности последней покупки, определить профиль каждого из кластеров.

**Основные цели проекта:**
1. Произвести предобработку исходного набора данных о транзакциях.
2. Провести разведывательный анализ данных и выявить основные закономерности.
3. Сформировать набор данных о характеристиках каждого из уникальных клиентов.
4. Построить несколько моделей машинного обучения, решающих задачу кластеризации клиентов, определить количество кластеров и проинтерпретировать их.
5. Спроектировать процесс предсказания категории интересов клиента и протестировать вашу модель на новых клиентах.




## Данные и их описание

Данные представляют собой таблицу в формате CSV, в каждой строке которой содержится информация об уникальной транзакции.

Признаки, описывающие каждую транзакцию:

* InvoiceNo — номер счёта-фактуры (уникальный номинальный шестизначный номер, присваиваемый каждой транзакции; буква "C" в начале кода указывает на отмену транзакции);
* StockCode — код товара (уникальное пятизначное целое число, присваиваемое каждому отдельному товару);
* Description — название товара;
* Quantity — количество каждого товара за транзакцию;
* InvoiceDate — дата и время выставления счёта/проведения транзакции;
* UnitPrice — цена за единицу товара в фунтах стерлингов;
* CustomerID — идентификатор клиента (уникальный пятизначный номер, однозначно присваиваемый каждому клиенту);
* Country — название страны, в которой проживает клиент.



Импорт базовых библиотек:

In [1]:
import pandas as pd
import numpy as np

import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots

import warnings 
from IPython.display import display, HTML

warnings.filterwarnings("ignore")

from sklearn.preprocessing import RobustScaler, MinMaxScaler, StandardScaler, OneHotEncoder
from sklearn.cluster import AgglomerativeClustering, KMeans, DBSCAN
from sklearn.metrics import silhouette_score
from sklearn.mixture import GaussianMixture
from sklearn.compose import make_column_transformer
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.manifold import TSNE
from countryinfo import CountryInfo
from geopy.distance import geodesic as GD
from category_encoders import BinaryEncoder

## 1. Знакомство со структурой данных

Первым делом необходимо понять, с какими данными предстоит работать, и произвести базовую предобработку данных — перевести признаки в необходимые для дальнейшей работы форматы.

Познакомьтесь с исходными данными поближе:

* Проведите статистический анализ исходных данных, посмотрев на основные диапазоны исходных признаков.
* Узнайте, сколько уникальных клиентов совершали транзакции в указанный период.
* Узнайте, из каких стран совершались транзакции.
* Исследуйте данные на наличие пропусков и дубликатов.
* Переведите столбцы в корректные форматы (например, даты в формат datetime).

In [66]:
data = pd.read_csv(
    "./data/data.csv", 
    encoding="ISO-8859-1", 
    dtype={'CustomerID': str,'InvoiceID': str}
)
print('Data shape: {}'.format(data.shape))
data.head()

Data shape: (541909, 8)


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850,United Kingdom


Посмотрим, с каким количеством уникальных клиентов мы имеем дело:

In [67]:
data['CustomerID'].nunique()

4372

А также на страны, из которых были оформлены заказы:

In [68]:
data['Country'].value_counts().nlargest(15)

Country
United Kingdom     495478
Germany              9495
France               8557
EIRE                 8196
Spain                2533
Netherlands          2371
Belgium              2069
Switzerland          2002
Portugal             1519
Australia            1259
Norway               1086
Italy                 803
Channel Islands       758
Finland               695
Cyprus                622
Name: count, dtype: int64

Мы видим, что перечень стран , из которых совершались транзакции, достаточно разнообразен  и представлен всеми континентами. Однако основная масса покупок совершена британцами.

Выясним период времени, за который были совершены транзакции:

In [69]:
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'])
print('Date interval {} to {}'.format(data['InvoiceDate'].dt.date.min(), data['InvoiceDate'].dt.date.max()))

Date interval 2010-12-01 to 2011-12-09


## 2. Преобразование, очистка и анализ данных

### 2.1. Преобразование и очистка данных о транзакциях

#### 2.1.1 Пропуски

Пропуски в столбце с идентификатором клиента (CustomerID) и описанием товара свидетельствуют о некорректных/незавершённых транзакциях. Удалите их из данных.

**Примечание.** Если посмотреть на распределение пропусков в столбцах Description и CustomerID, то можно заметить, что достаточно удалить строки, содержащие пропуски в столбце CustomerID, тогда пропуски в столбце Description удаляются автоматически.


In [70]:
data = data.dropna()

#### 2.1.2. Дубликаты

Проверьте данные на наличие дубликатов. Удалите их из данных.


In [71]:
data = data.drop_duplicates()
print(f'Количество дубликатов в данных: {data.duplicated().sum()}')

Количество дубликатов в данных: 0


#### 2.1.3. Транзакции с отрицательным количеством товара

Из приведённых выше инсайтов о данных мы узнали все особенности отмены заказов. Когда мы будем сегментировать клиентов, мы будем определять их покупательскую способность. При расчёте покупательской способности нам будет очень важно учесть возвраты этого клиента.

Чтобы подсчитать количество возвратов, для начала нам надо определить, сколько уникальных товаров указано в транзакции (корзине) для каждой уникальной пары «клиент — заказ»:


In [72]:
temp = data.groupby(by=['CustomerID', 'InvoiceNo'], as_index=False)['InvoiceDate'].count()
nb_products_per_basket = temp.rename(columns = {'InvoiceDate':'Number of products'})
nb_products_per_basket.head()

,CustomerID,InvoiceNo,Number of products
0,12346,541431,1
1,12346,C541433,1
2,12347,537626,31
3,12347,542237,29
4,12347,549222,24


**Примечание.** Более 16 % уникальных заказов являются возвратами. Интересный факт: если мы подсчитали количество транзакций, содержащих признак возврата, в изначальной таблице, где на каждый уникальный товар заведена отдельная строка, то мы получили бы, что количество возвратов менее 1 %. Однако это число было бы некорректным.

Создайте в ваших данных о транзакциях признак `QuantityCanceled`. Этот признак будет указывать на количество отменённого впоследствии товара для каждой транзакции, на которую найдётся противоположная ей транзакция с возвратом. Для транзакций, для которых не было возвратов, признак будет равен 0. Для транзакций, для которых указано отрицательное количество товара, но в данных отсутствует контрагент, признак будет пустым.

В качестве вспомогательного инструмента мы подготовили для вас функцию `get_quantity_canceled()`. Эта функция принимает на вход таблицу с транзакциями и возвращает объект `Series` — столбец, в котором указано количество отменённого впоследствии товара для каждой транзакции. Если транзакция не имеет контрагента, этот признак помечается как `NaN`.

Отметим, что эта функция не учитывает сложный случай, когда количество отменённого товара в транзакции-возврате больше, чем количество товара, которое указано в любой из отдельных транзакций на покупку (это случай, когда клиент сделал несколько заказов одного и того же товара, а потом оформил возврат на все товары разом). При желании вы можете самостоятельно модифицировать функцию для предобработки, чтобы учесть этот пограничный случай.

*Осторожно! Поиск отменённых транзакций занимает много времени из-за большого количества строк в таблице. Выполнение следующего кода может занять до 20 минут в зависимости от мощности компьютера.*

In [73]:
def get_quantity_canceled(data):
    """Функция для создания признака количества отменённых заказов. 
    Функция принимает на вход таблицу и возвращает столбец, в котором указано количество отменённого впоследствии товара для кажой транзакции.
    Если транзакция с отрицательным количеством товара не имеет контрагента, данный признак помечается как NaN.

    Args:
        data (DataFrame): таблица с транзакциями

    Returns:
        Series: столбец с количеством отменённого товара
    """
    # Инициализируем нулями Series той же длины, что и столбцы таблицы.
    quantity_canceled = pd.Series(np.zeros(data.shape[0]), index=data.index)    
    negative_quantity = data[(data['Quantity'] < 0)].copy()
    for index, col in negative_quantity.iterrows():
        # Создаём DataFrame из всех контрагентов
        df_test = data[(data['CustomerID'] == col['CustomerID']) &
                       (data['StockCode']  == col['StockCode']) & 
                       (data['InvoiceDate'] < col['InvoiceDate']) & 
                       (data['Quantity'] > 0)].copy()
        # Транзация-возврат не имеет контрагента — ничего не делаем
        if (df_test.shape[0] == 0): 
            # Помечаем столбец как пропуск
            quantity_canceled.loc[index] = np.nan
        # Транзакция-возврат имеет ровно одного контрагента
        # Добавляем количество отменённого товара в столбец QuantityCanceled 
        elif (df_test.shape[0] == 1): 
            index_order = df_test.index[0]
            quantity_canceled.loc[index_order] = -col['Quantity']       
        # Транзакция-возврат имеет несколько контрагентов
        # Задаём количество отменённого товара в столбец QuantityCanceled для той транзакции на покупку,
        # в которой количество товара больше количества товаров в транзакции-возврате.
        elif (df_test.shape[0] > 1): 
            df_test.sort_index(axis=0 ,ascending=False, inplace = True)        
            for ind, val in df_test.iterrows():
                if val['Quantity'] < -col['Quantity']: 
                    quantity_canceled.loc[ind] = 'x' # отлавливаем пограничный случай , при котором количество 
                                                     # отменённого товара в транзакции-возврате больше, чем количество товара,
                                                     # которое указано в любой из отдельных транзакций на покупку
                quantity_canceled.loc[ind] = -col['Quantity']
                break    
    return quantity_canceled

data['QuantityCanceled'] = get_quantity_canceled(data)

print(f'Количество пограничных случаев: {data[data["QuantityCanceled"] == "x"].shape[0]}')


Количество пограничных случаев: 0


Узнайте, сколько транзакций в данных не имеют контрагентов, и, если их количество невелико, удалите их из данных.

Когда вы разобрались с возвратами, удалите транзакции с отрицательным количеством товара — они нам больше не нужны.

In [74]:
num_emty_canceled = data[data['QuantityCanceled'].isna()].shape[0]
print(f'Количество возвратных транзакций, не имеющих контрагентов: {num_emty_canceled}')

# количество таких транзакций невелико, поэтому удаляем их из наших данных
data = data.dropna()
# удаляем данные с отрицательным количеством товара
data = data[data['Quantity'] >= 0]

Количество возвратных транзакций, не имеющих контрагентов: 1303


#### 2.1.4. Специализированные транзакции

Следующая задача — обработать специальные виды транзакций, которые обозначаются латинскими буквами.

В данных присутствует шесть специальных видов транзакций. С помощью регулярных выражений найдите такие коды товаров (StockCode), которые начинаются с латинских букв (при этом коды могут содержать цифры).

**Подсказка.** В качестве шаблона для поиска используйте строку '^[a-zA-Z]+'.

Чтобы понять, что означают эти коды, можно заглянуть в столбец с описанием (Description), например POST означает почтовые расходы, C2 — расходы на транспортировку, BANK CHARGES — банковские расходы.

Специальные операции не характеризуют покупательскую способность клиентов, так как не относятся напрямую к их покупкам, поэтому такие записи нам не нужны. Удалите все специальные транзакции из таблицы.

In [75]:

data = data[data['StockCode'].str.match(r'^[a-zA-Z]+') == False]

#### 2.1.5. Транзакции с товарами без стоимости

При просмотре описательных статистик можно заметить, что на некоторые товары установлена цена в 0 фунтов стерлингов. Таких транзакций оказывается менее 1 % — можно удалить их.

In [76]:
data = data[data['UnitPrice'] > 0]
data = data.reset_index(drop=True)

In [77]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 391150 entries, 0 to 391149
Data columns (total 9 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   InvoiceNo         391150 non-null  object        
 1   StockCode         391150 non-null  object        
 2   Description       391150 non-null  object        
 3   Quantity          391150 non-null  int64         
 4   InvoiceDate       391150 non-null  datetime64[ns]
 5   UnitPrice         391150 non-null  float64       
 6   CustomerID        391150 non-null  object        
 7   Country           391150 non-null  object        
 8   QuantityCanceled  391150 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(6)
memory usage: 26.9+ MB


#### 2.1.6. Общая стоимость товаров в транзакции

Добавьте в ваш датасет общую цену заказа (TotalPrice). Она рассчитывается как:
 
 **общая цена = цена за единицу товара * (количество товаров в заказе - количество возвращённых товаров).**

Этот признак впоследствии поможет вам рассчитать покупательскую способность каждого из клиентов.

In [78]:
data['TotalPrice'] = data['UnitPrice'] * (data['Quantity'] - data['QuantityCanceled'])

data['TotalPrice'] = data['TotalPrice'].astype('float')
data['QuantityCanceled'] = data['QuantityCanceled'].astype('int')

Помимо рекомендованных преобразований, вы также можете придумать собственные и применить их к исходным данным.
После завершения предобработки сохраните результат очищения данных в отдельный файл, чтобы впоследствии вам не приходилось повторять эти действия.


In [79]:
# извлекаем новые признаки из дыты оформления заказа
data['week_day'] = data['InvoiceDate'].dt.day_of_week
data['month'] = data['InvoiceDate'].dt.month
data['hour'] = data['InvoiceDate'].dt.hour
data['day_in_month'] = data['InvoiceDate'].dt.day
data['date'] = data['InvoiceDate'].dt.date
data['quarter'] = data['InvoiceDate'].dt.quarter

# также извлекаем новые признаки из страны, из которой был оформлен заказ

# прежде всего избавимся от клиентов с неопределенной геолокацией
mask1 = data['Country'] != 'Unspecified'
mask2 = data['Country'] != 'European Community'
data = data[mask1 & mask2]
data = data.reset_index(drop=True)

country_list = list(data['Country'].value_counts().index) # список всех стран в датасете
country_list.remove('Channel Islands') # удаляем те страны, которые не знает countryinfo, данные позже найдем вручную
country_list.remove('EIRE') 

country_dict = dict()         # словарь для геоданных
for country in country_list:  # заполняем словарь в цикле по списку стран
    country_info = CountryInfo(country).info()
    country_dict[country] = [country_info['capital_latlng'], # координаты столицы
                             country_info['subregion'],      # регион
                             country_info['languages'][0],   # основной язык
                             country_info['population']]               # население
    
country_dict['Channel Islands'] = [[49.185833, -2.11], 'Northern Europe', 'en', 163857] # заполняем "руками"
country_dict['EIRE'] = [[53.349764, -6.260273], 'Northern Europe', 'ga', 5123536]

capital_dist = dict()  # словарь расстояний между столицей государства и столицей Великобритании
for country in country_dict.keys():
    capital_dist[country] = round(GD(country_dict[country][0], country_dict['United Kingdom'][0]).km)


n_uniq_cust = data['CustomerID'].nunique() # число уникальных клиентов в датасете
# находим число уникальных клиентов для каждой страны
count_uniq_cust = data.groupby(by='Country')['CustomerID'].nunique().sort_values(ascending=False) 

# новые признаки
data['n_uniq_cust_country'] = data['Country'].apply(lambda x: count_uniq_cust.loc[x]) # число уникальных клиентов 
# процент уникальных клиентов в стране по отношению к общему числу уникальных клиентов 
data['uniq_custID_ratio'] = data['n_uniq_cust_country'] / n_uniq_cust * 100           
data['country_dist'] = data['Country'].apply(lambda x: capital_dist[x])      # расстояние от столицы государства до Лондона
data['region'] = data['Country'].apply(lambda x: country_dict[x][1])     # регион на карте мира
data['language'] = data['Country'].apply(lambda x: country_dict[x][2])   # основной язык населения страны
data['population'] = data['Country'].apply(lambda x: country_dict[x][3]) # население


In [80]:
# сохраняем результаты предобработки
data.to_csv('data/new_data.csv', index= False)

In [81]:
# data_2 = pd.read_csv('data/new_data.csv')
# data_2['InvoiceDate'] = pd.to_datetime(data_2['InvoiceDate'])
# data = data_2

### 2.2. Разведывательный анализ

После предобработки исходных данных произведите разведывательный анализ и исследуйте транзакции, ответив на следующие вопросы:

* Клиенты из каких стран покупают больше и чаще?
* Какие страны приносят наибольшую сезонную выручку?
* Присутствует ли в продажах сезонность (когда покупают чаще)?
* Сгруппируйте данные по датам и часам совершения транзакции и найдите количество заказов на каждый день-час. Затем найдите среднее количество ежедневно поступающих заказов в каждый из часов.
* Каково распределение среднего количества ежедневно поступающих заказов по времени суток (часу совершения транзакции)? 

**Примечание.** Вы можете сформулировать и другие вопросы. Главная цель — извлечь максимум понятной информации из исходных данных.

Свои рассуждения сопроводите графиками и диаграммами.



In [82]:
# построим графики, отражающие зависимость общего количества заказов и 
# суммарной выручки в разрезе временных характеристик
dt_features = ['week_day', 'month', 'hour', 'day_in_month']

for i, feature in enumerate(dt_features):
    
    group = data.groupby(by=feature)[['TotalPrice', 'InvoiceDate']].agg({'TotalPrice' : 'sum',
                                                                         'InvoiceDate' : 'count'})
    fig = make_subplots( rows=2, cols=1 )
    fig.add_trace(go.Bar(x=group.index,
                     y=group['TotalPrice'],
                     name='Суммарная стоимость заказов'),
                     row=1, col=1)
                     
    fig.add_trace(go.Bar(x=group.index,
                     y=group['InvoiceDate'],
                     name='Общее количество заказов'),
                     row=2, col=1, )
    fig.update_layout(width=800, xaxis_title=feature,
                  title_text=f'Суммарная стоимость и общее кол-во заказов  в разрезе признака {feature}')
    fig.write_html(f'plotly/bar_{i}.html')
    fig.show()

Исходя из построенных графиков можно сделать следующие выводы:
* суммарная стоимость заказов и их общее количество коррелируют между собой в разрезе любого признака
* исследуемые показатели достигают максимальных значений в следующие временные промежутки: час- 12, день недели - четверг, месяц - ноябрь, квартал - 4
* по субботтам , а так же с 21 вечера до 6 утра в остальные дни заказы не принимаются

In [83]:
# посмотрим на распределение количества заказов по часам 
date_hour_pivot = data.pivot_table(values='InvoiceDate',index='date', columns='hour', aggfunc='count', fill_value=0)
date_hour_mean = date_hour_pivot.mean()

fig = px.histogram(date_hour_mean, 
                   x=date_hour_mean.index,
                   y=date_hour_mean.values, 
                   nbins=24, 
                   width=800,
                   labels={'y': 'Среднее количество заказов'},
                   title='Распределение среднего количества ежедневно поступающих заказов по часам')
fig.write_html('plotly/hist_1.html')
fig.show()

Распределение среднего количества ежедневно поступающих заказов по часам близко к нормальному. Максимальное количество заказов оформляется с 12 до 13

In [84]:
# сводная таблица суммарной стоимости заказов по странам по кварталам (без Британии)
quarter_country_pivot = pd.pivot_table(data=data[data['Country'] != 'United Kingdom'], 
                                       index='quarter', 
                                       columns='Country', 
                                       values='TotalPrice', 
                                       aggfunc='sum', 
                                       fill_value=0)

fig = px.imshow(quarter_country_pivot,
                title='Тепловая карта сезонной выручки по странам (без Британии)')
fig.update_xaxes(tickangle=45)
fig.write_html('plotly/heatmap_1.html')
fig.show()

После Великобритании в лидерах как по общей суммарной, так и по сезонной суммарной выручке пять стран : Нидерланды, Германия, Франция, Ирландия и Австралия

### 2.3. Построение RFM-таблицы и поиск RFM-выбросов

Мы добрались до самой интересной части нашей задачи. Нам предстоит сформировать признаки, на основе которых мы будем производить сегментацию клиентов.

Для этого давайте познакомимся с очень популярным методом для анализа потребительской ценности под названием RFM. 

<center> <img src=https://miro.medium.com/max/1400/1*uYQjy9SUjW7iWHc2gGanQQ.png align="right" width="400"/> </center>

Метод заключается в группировке клиентов на основе следующих параметров:
* Recency (Давность) — давность последней покупки клиента;
* Frequency (Частота) — общее количество покупок клиента;
* Monetary Value (Денежная ценность) — сколько денег потратил клиент.


Суть RFM-анализа состоит в том, что мы разделяем всех клиентов на группы в зависимости от того, как давно они сделали последнюю покупку, как часто покупали и насколько большой была сумма их заказов. 

Например, вот так может выглядеть интерпретация кластеров для случая RF-сегментации (анализа на основе давности и частоты заказов клиента):

<img src=https://retailrocket.ru/wp-content/uploads/2017/06/rfm-1.png>

Задача маркетологов — вести клиента в зону лояльных.

Мы можем рассчитать RFM-характеристики для каждого из клиентов в нашем датасете и на их основе с помощью методов кластеризации построить подобные сегменты клиентов, привязанные к нашим данным.

Чтобы получить RFM-таблицу, нам необходимо сгруппировать данные по идентификаторам клиента и рассчитать следующие  агрегированные характеристики:

* Recency для i-го клиента рассчитывается как разница между датой и временем последнего заказа и точкой отсчёта, переведённая в дни:
    $$t_0-max(t_{i1}, t_{i2},..., t_{iM})$$

    где $t_{ij}$ — дата и время совершения i-ым клиентом своей j-ой покупки.

    В качестве точки отсчёта $t_0$ берём дату на один день «старше», чем все наши данные. Это будет 10 декабря 2011 года (в формате datetime — '2011-12-10 00:00:00').

* Frequency рассчитывается как общее количество уникальных заказов, которые совершил i-ый клиент.
* Monetary Value рассчитывается как общая сумма денег, которую i-ый клиент потратил на наши товары (с учётом возвратов).

Когда вы рассчитаете все характеристики, не забудьте дать столбцам результирующей таблицы соответствующие названия.

In [85]:
day_d = pd.to_datetime('2011-12-10 00:00:00') # временная метка для расчета срока давности заказов

rfm_table = data.groupby(by='CustomerID')[['InvoiceDate', 
                                            'InvoiceNo', 
                                            'TotalPrice']].agg({'InvoiceDate' : 'max',
                                                                'InvoiceNo' : 'nunique',
                                                                'TotalPrice' : 'sum' })
                                                        
rfm_table['Recency'] = (day_d - rfm_table['InvoiceDate']).dt.days  
rfm_table.rename(columns={'InvoiceNo' : 'Frequency', 'TotalPrice' : 'Monetary'}, inplace=True) 
rfm_table.drop('InvoiceDate', axis=1, inplace=True)   
rfm_table.info()                                                  

<class 'pandas.core.frame.DataFrame'>
Index: 4329 entries, 12346 to 18287
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Frequency  4329 non-null   int64  
 1   Monetary   4329 non-null   float64
 2   Recency    4329 non-null   int64  
dtypes: float64(1), int64(2)
memory usage: 135.3+ KB


После того как вы подготовите RFM-таблицу, визуализируйте каждую из трёх компонент, например, в виде коробчатых диаграмм (boxplot). Вы получите примерно следующие результаты (графики могут отличаться в зависимости от того, как вы выполните предобработку данных):

In [86]:
boxes = [px.box(rfm_table, x=column) for column in rfm_table.columns]
fig = make_subplots(rows=1,
                    cols=3, 
                    subplot_titles=("Frequency", "Monetary", "Recency"))

for i, box in enumerate(boxes):
    fig.add_trace(boxes[i]['data'][0], row=1, col=i+1)

fig.update_layout(showlegend=True, title='Коробчатые диаграммы признаков RFM-таблицы')
fig.write_html('plotly/boxplot_1.html')
fig.show()

Что интересного здесь можно увидеть? Есть клиенты с аномально большим количеством сделанных заказов (более 100 штук), а также клиенты, общая стоимость заказов которых превышает 190 тысяч фунтов стерлингов.

Чем это плохо? Выбросы могут отрицательно сказаться на результатах работы методов кластеризации, неустойчивых к ним, например алгоритма KMeans, поэтому хотелось бы от них избавиться. Однако терять много ценных данных о клиентах тоже не хочется, поэтому ограничимся верхней границей соответствующей квантили уровня 0.95. Таким образом, мы удалим данные тех клиентов, для которых значение параметра Frequency или параметра Monetary выше, чем у 95 % клиентов.


In [87]:
q_freq = rfm_table['Frequency'].quantile(0.95)
q_monet = rfm_table['Monetary'].quantile(0.95)

mask_1 = rfm_table['Frequency'] < q_freq
mask_2 = rfm_table['Monetary'] < q_monet
rfm_table = rfm_table[mask_1 & mask_2]

rfm_table.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4020 entries, 12346 to 18287
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Frequency  4020 non-null   int64  
 1   Monetary   4020 non-null   float64
 2   Recency    4020 non-null   int64  
dtypes: float64(1), int64(2)
memory usage: 125.6+ KB


## 3. Моделирование и оценка качества моделей

### 3.1. Кластеризация на основе RFM-характеристик

Чтобы результаты кластеризации было удобнее интерпретировать, вы можете уменьшить размерность исходных признаков до двух компонент.

**Подсказка.** Чтобы методы понижения размерности работали стабильно, данные необходимо стандартизировать/нормализовать. Для удобства оберните эти шаги по предобработке данных в pipeline.

Произведите предобработку исходных данных. На основе RFM-признаков кластеризуйте клиентов онлайн-магазина подарков с помощью известных вам методов (используйте минимум три метода).

Подберите оптимальное количество кластеров для выбранных методов с помощью коэффициента силуэта, перебирая возможные значения от 3 до 10 включительно (большее или меньшее количество кластеров будет нерелеватно для маркетинга). 

Выберите алгоритм с наибольшим коэффициентом силуэта, сделайте предсказание меток кластеров.


In [88]:
# создаем вспомогательные функции
def get_silhouette_agg(cluster_num, X):
    """Функция для расчета коэффициента силуэта результатов кластеризации,
    полученных с помощью AgglomerativeClustering библиотеки sklearn

    Args:
        cluster_num (int): количество кластеров
        X (dataframe): данные для обучения модели кластеризации

    Returns:
        float: найденное значение коэффициента для заданного количества кластеров
    """
    agg =  AgglomerativeClustering(n_clusters=cluster_num, linkage='average', metric='l2')
    agg.fit(X)
    silhouette_agg = silhouette_score(X, agg.labels_)
    return silhouette_agg


def get_silhouette_km(cluster_num, X):
    """Функция для расчета коэффициента силуэта результатов кластеризации,
    полученных с помощью KMeans библиотеки sklearn

    Args:
        cluster_num (int): количество кластеров
        X (dataframe): данные для обучения модели кластеризации

    Returns:
        float: найденное значение коэффициента для заданного количества кластеров
    """
    k_means =  KMeans(n_clusters=cluster_num, random_state=42, init='random')
    k_means.fit(X)
    silhouette_km = silhouette_score(X, k_means.labels_)
    return silhouette_km


def get_silhouette_gm(cluster_num, X):
    """Функция для расчета коэффициента силуэта результатов кластеризации,
    полученных с помощью GaussianMixture библиотеки sklearn

    Args:
        cluster_num (int): количество кластеров
        X (dataframe): данные для обучения модели кластеризации

    Returns:
        float: найденное значение коэффициента для заданного количества кластеров
    """
    gm =  GaussianMixture(n_components=cluster_num, random_state=42)
    silhouette_gm = silhouette_score(X, gm.fit_predict(X))
    return silhouette_gm


def get_silhouette_res_and_scatter(scaler, enc_array, index_data, p, n):
    """функция возвращает результат перебора всех вариантов сочетаний 
    способ понижения размерности - метод кластеризации - количество кластеров,
    отсортированный по величине коэффициента силуэта;
    также функция строит диаграммы рассеяния данных после понижения размерности

    Args:
        scaler (class): способ преобразования данных данных
        enc_array (array): данные для преобразования
        index_data (dataframe): берем из этого датафрейма индексы 
        p (float): параметр perplexity TNSE
        n (int): для нумерации сохраняемых изображений
      

    Returns:
        list: список,  состоящий из отсортированных по коэффициенту силуэта результатов перебора, 
        словаря с данными пониженной размерности и массива  нормальзованных данных
    """
    
    scale_array = scaler.fit_transform(enc_array) # преобразовываем данные
    # понижаем их размерность тремя способами
    tsne = TSNE(n_components=2, perplexity=p, n_iter=251, random_state=42)
    country_rfm_tsne = tsne.fit_transform(scale_array)

    pca = PCA(n_components=2, random_state=42)
    country_rfm_pca = pca.fit_transform(scale_array)

    svd = TruncatedSVD(n_components=2, n_iter=500, random_state=42)
    country_rfm_svd = svd.fit_transform(scale_array)

    country_rfm_tsne = pd.DataFrame(country_rfm_tsne, columns=['component_1', 'component_2'], index=index_data.index)
    country_rfm_pca = pd.DataFrame(country_rfm_pca, columns=['component_1', 'component_2'], index=index_data.index)
    country_rfm_svd = pd.DataFrame(country_rfm_svd, columns=['component_1', 'component_2'], index=index_data.index)
    
    # визуализируем данные после понижения размерности
    scatters = [px.scatter(country_rfm_tsne, x='component_1', y='component_2'), 
                px.scatter(country_rfm_pca, x='component_1', y='component_2'),
                px.scatter(country_rfm_svd, x='component_1', y='component_2')]
    fig = make_subplots(rows=1, cols=3, 
                        subplot_titles=("t-SNE", "PCA", "SVD"))

    for i, box in enumerate(scatters):
        fig.add_trace(scatters[i]['data'][0], row=1, col=i+1)
    fig.update_layout(showlegend=True, 
                title=f'Диаграммы рассеяния данных, полученных различными методами понижения размерности', 
                yaxis_title='component_2')
    fig.update_xaxes(title='component_1', col=1, row=1)
    fig.update_xaxes(title='component_1', col=2, row=1)
    fig.update_xaxes(title='component_1', col=3, row=1)
    fig.write_html(f'plotly/scatter_{n}.html')
    fig.show()
        
    reduction_dict = {'tsne' : country_rfm_tsne, # словарь с данными с пониженной размерностью
                      'pca' : country_rfm_pca, 
                      'svd' : country_rfm_svd} 
    
    model_dict = {'agg' : get_silhouette_agg, # словарь с функциями 
                  'km': get_silhouette_km, 
                  'gm' : get_silhouette_gm} 

    res_sil = {"reduction" : [], # словарь для сохранения результатов перебора
               "model" : [],
               "silhouette": [], 
               "cluster": []} 

 
    for data_key in reduction_dict.keys(): # перебираем в цикле данные
        for model_key in model_dict.keys(): # перебираем в цикле функции
            for cluster_num in range(3, 10): # перебираем в цикле количество кластеров
                res_sil['silhouette'].append(model_dict[model_key](cluster_num, reduction_dict[data_key]))
                res_sil["cluster"].append(cluster_num)
                res_sil['reduction'].append(data_key)
                res_sil['model'].append(model_key)

    res_sil_df = pd.DataFrame(res_sil)
    res_sil_df.sort_values(by='silhouette', ascending=False, inplace=True)
    
    return [res_sil_df, reduction_dict, scale_array]

In [89]:
# Подбираем оптимальное количество кластеров клиентов различными методами кластеризации для каждого варианта сниженя размерности
rfm_res_list = get_silhouette_res_and_scatter(scaler=MinMaxScaler(),enc_array=rfm_table, index_data=rfm_table ,p=12, n=1)
print('Результаты подбора метода кластеризации клиентов по rfm-признакам ')
print(rfm_res_list[0].head(15))

Результаты подбора метода кластеризации клиентов по rfm-признакам 
   reduction model  silhouette  cluster
28       pca    km    0.535308        3
49       svd    km    0.534958        3
43       svd   agg    0.486057        4
7       tsne    km    0.477131        3
44       svd   agg    0.476677        5
14      tsne    gm    0.475980        3
29       pca    km    0.474448        4
50       svd    km    0.473386        4
45       svd   agg    0.471269        6
30       pca    km    0.466722        5
51       svd    km    0.465130        5
46       svd   agg    0.464526        7
13      tsne    km    0.459424        9
35       pca    gm    0.456981        3
56       svd    gm    0.454454        3


На диаграмме рассеяния данных, полученных методом стохастического вложения соседей с t-распределением, можно различить кластеры сложной формы. Имеет смысл попробовать применить кластеризацию на основе плотности.

In [90]:
def get_silhouette_db(X, e, ms):
    """Функция для расчета коэффициента силуэта результатов кластеризации,
    полученных с помощью метода  на основе плотности DBSCAN библиотеки sklearn

    Args:
        X (dataframe): данные для обучения модели кластеризации
        e (float): параметр eps, максимальное расстояние между двумя точками, 
                   при котором точки относятся к одному кластеру
        ms (int): параметр min_samples, количество точек в окрестности точки, 
                  которая будет считаться базовой точкой
    Returns:
        list: список, состоящий из значения коэффициента силуэта и количества кластеров
    """
    db =  DBSCAN(eps=e, min_samples=ms, n_jobs=-1)
    db.fit(X)
    silhouette_db = silhouette_score(X, db.labels_)
    return [silhouette_db, len(np.unique(db.labels_)) -1]


res_sil = {"silhouette": [], # словарь для сохранеия результатов перебора параметров
           "cluster": [], 
           'eps' : [], 
           'ms' : [], 
           'data' : []} 


for data_key in rfm_res_list[1].keys(): # перебираем в цикле данные 
    for e in np.arange(0.01,0.1,0.02): # перебираем в цикле значения параметра eps
        for ms in np.arange(2, 6, 1): # перебираем в цикле значение параметра min_samples 
            res_sil['silhouette'].append(get_silhouette_db( rfm_res_list[1][data_key], e, ms)[0])
            res_sil["cluster"].append(get_silhouette_db(rfm_res_list[1][data_key], e, ms)[1])
            res_sil['eps'].append(e)
            res_sil['ms'].append(ms)
            res_sil['data'].append(data_key)
        
    
res_sil_df = pd.DataFrame(res_sil)
res_sil_df = res_sil_df[res_sil_df['cluster'] >=3].sort_values(by='silhouette', ascending=False)
print('Результаты подбора параметров кластеризации методом DBSCAN')
print(res_sil_df.head(10))

Результаты подбора параметров кластеризации методом DBSCAN
    silhouette  cluster   eps  ms  data
16    0.355660      415  0.09   2  tsne
12    0.338066      541  0.07   2  tsne
17    0.314449      298  0.09   3  tsne
18    0.278856      250  0.09   4  tsne
13    0.251684      349  0.07   3  tsne
8     0.217488      663  0.05   2  tsne
19    0.201444      216  0.09   5  tsne
49    0.198559        3  0.05   3   svd
53    0.183227        3  0.07   3   svd
52    0.183227        3  0.07   2   svd


Применение  DBSCAN оказалось неэффективным.

Получаем метки кластеров с помощью модели, показавшей максимальное значение коэффициента силуэта

In [91]:
rfm_table_scale = pd.DataFrame(rfm_res_list[2], columns=['Frequency', 'Monetary', 'Recency'], 
                               index=rfm_table.index)

km = KMeans(n_clusters=3, random_state=42)
km.fit(rfm_res_list[1]['pca'])

rfm_table_scale['pca_km_cluster'] = km.labels_
print('Количество клиентов в кластерах:')
rfm_table_scale['pca_km_cluster'].value_counts()


Количество клиентов в кластерах:


pca_km_cluster
2    2208
1    1016
0     796
Name: count, dtype: int64

### 3.2. Интерпретация результатов кластеризации

Перейдём к интерпретации полученных кластеров.

#### 3.2.1. Визуализация кластеров

Визуализируйте результаты в виде 3D-диаграммы с осями Recency, Frequency и Monetary. Проанализируйте полученную диаграмму и попробуйте понять, какие кластеры у вас получились.

In [92]:
rfm_table_scale['pca_km_cluster'] = rfm_table_scale['pca_km_cluster'].astype(str)

fig = px.scatter_3d(rfm_table_scale, 
                    x='Frequency', 
                    y='Monetary', 
                    z='Recency', 
                    color='pca_km_cluster',
                    height=700) 
fig.update_traces(marker_size = 3)
fig.update_layout(
    title={'text':'Диаграмма рассеяния RFM-данных.'})
fig.write_html('plotly/3dscatter_1.html')
fig.show()

#### 3.2.2. Построение профиля кластеров

Далее составьте так называемый профиль кластеров. Для этого вам необходимо вернуться от декомпозированных данных (если вы производили понижение размерности) к RFM-таблице (очищенной от выбросов).

Сгруппируйте RFM-таблицу по полученным кластерам и рассчитайте среднее по каждому из признаков.

Чтобы результаты было проще интерпретировать, давайте познакомимся с одним из способов визуализации профиля кластеров — **Radar Chart** (полярная диаграмма, или диаграмма паутины). Это графическое представление значений нескольких эквивалентных категорий в форме паутины.

Radar Chart часто используется при определении профиля кластеров. На концах паутины откладываются оси, соответствующие признакам, описывающим объекты. На каждой из осей для каждого кластера откладываются средние значения соответствующих характеристик. Соединив точки по осям, мы получаем многоугольник. 

Пример полярной диаграммы для задачи кластеризации учеников по интересам:

<img src=https://www.datanovia.com/en/wp-content/uploads/2020/12/radar-chart-in-r-customized-fmstb-radar-chart-1.png width=500>

На этой диаграмме мы видим визуализацию признаков для одного из кластеров. Видно, что ученики, принадлежащие этому кластеру, в большей степени увлекаются музыкой (Music), а в меньшей — программированием (Programm).

В модуле `graph_objects` библиотеки `plotly` есть встроенная функция `Scatterpolar`, которая позволяет построить полярную диаграмму. На основе этой функции мы подготовили для вас функцию `plot_cluster_profile()`, которая позволяет визуализировать профиль каждого из кластеров в виде полярной диаграммы. У неё есть два параметра: `grouped_data` — сгруппированные по кластерам характеристики объектов (клиентов), `n_clusters` — количество кластеров.

Главное условие использования полярной диаграммы — все признаки должны быть приведены к единому масштабу с помощью нормализации, где 1 будет означать максимум, а 0 — минимум. Шаг с нормализацией мы также добавили в функцию `plot_cluster_profile()`.


In [93]:
def plot_cluster_profile(grouped_data, n_clusters, m):
    """Функция для визуализации профиля кластеров в виде полярной диаграммы.

    Args:
        grouped_data (DataFrame): таблица, сгруппированная по номерам кластеров с агрегированными характеристиками объектов.
        n_clusters (int): количество кластеров
        title (str): Название диаграммы
        m (int): для нумерации изображений
    """
    # Нормализуем сгруппированные данные, приводя их к масштабу 0-1.
    scaler = MinMaxScaler()
    grouped_data = pd.DataFrame(scaler.fit_transform(grouped_data), columns=grouped_data.columns)
    # Создаём список признаков
    features = grouped_data.columns
    # Создаём пустую фигуру
    fig = go.Figure()
    # В цикле визуализируем полярную диаграмму для каждого кластера
    for i in range(n_clusters):
        # Создаём полярную диаграмму и добавляем её на общий график
        fig.add_trace(go.Scatterpolar(
            r=grouped_data.iloc[i].values, # радиусы
            theta=features, # название засечек
            fill='toself', # заливка многоугольника цветом
            name=f'Cluster {i}', # название — номер кластера
        ))
    # Обновляем параметры фигуры
    fig.update_layout(
        title = 'Полярная диаграмма кластеров',
        showlegend=True, # отображение легенды
        autosize=False, # устаналиваем свои размеры графика
        width=600, # ширина (в пикселях)
        height=600, # высота (в пикселях)
    )
    fig.write_html(f'plotly/scatterpolar_{m}.html')
    fig.show()

Теперь у вас есть удобный инструмент для анализа профиля кластеров. Воспользуйтесь им, чтобы проинтерпретировать результаты, полученные на предыдущем шаге.

In [94]:
group_rfm = rfm_table_scale.groupby(by='pca_km_cluster')[['Frequency', 'Monetary', 'Recency']].mean()
plot_cluster_profile(group_rfm, 3, m=1)

Попытаемся охарактеризовать полученные кластеры, опираясь на диаграмму рассеяния и профили кластеров :
* кластер 0 - самый малочисленный, кластер лояльных клиентов, покупающих часто и приносящих максимальную выручку
* кластер 2 - самый многочисленный, разовые покупки с небольшой суммой выручки, совершенные в недавнее время. Скорее всего именно на этот кластер иммеет смысл обратить внимание маркетологам с целью перехода как можно большего количества клиентов из этого кластера в кластер лояльных
* кластер 1 - кластер клиентов "в зоне потери", совершают покупки редко, приносят малую прибыль, давность совершения последней покупки - максимальная


## 4. Кластеризация на основе RFM-характеристик и дополнительных признаков

 Попробуем улучшить качество кластеризации, объеденив  RFM-признаки с признаками, полученными ранее на этапе разведывательного анализа, например с "геопризнаками". При этом мы увеличим размерность данных и рискуем сделать результаты кластеризации неинтерпретируемыми. Чтобы избежать этого попытаемся кластеризовать сами "геоданные", получив в итоге новый признак, указывающий на принадлежность клиента к определенному кластеру стран.

In [95]:
# удаляем из данных выбросы, выявленные при построении коробчатых диаграмм rfm-признаков
data['nan_id'] = data['CustomerID'].apply(lambda x: x if x in list(rfm_table.index) else np.nan)
data_cleaned = data.dropna()

In [96]:
country_data = data_cleaned[['Country', 'region', 'language', 'country_dist', 'uniq_custID_ratio', 'population']]
country_data = country_data.drop_duplicates().reset_index().drop('index', axis=1).set_index('Country')
country_data.head()

,region,language,country_dist,uniq_custID_ratio,population
Country,,,,,
United Kingdom,Northern Europe,en,0,90.459690,64105654
Netherlands,Western Europe,nl,358,0.207900,16881000
Germany,Western Europe,de,933,2.171402,80783000
France,Western Europe,fr,344,2.009702,66078000
Switzerland,Western Europe,de,749,0.485100,8183800


In [97]:
# подготовим данные для обучения
ct = make_column_transformer((OneHotEncoder(), ['region']),
                        (BinaryEncoder(), ['language']),
                        (MinMaxScaler(), ['country_dist', 'uniq_custID_ratio', 'population']))

enc_country_data = ct.fit_transform(country_data)

In [98]:
# подбираем способ кластеризации
res_country = get_silhouette_res_and_scatter(scaler=StandardScaler(), enc_array=enc_country_data, index_data=country_data, p=9, n=2)
print('Результаты подбора метода кластеризации стран: ')
print(res_country[0].head(15))

Результаты подбора метода кластеризации стран: 
   reduction model  silhouette  cluster
0       tsne   agg    0.648379        3
1       tsne   agg    0.646059        4
15      tsne    gm    0.646059        4
8       tsne    km    0.646059        4
14      tsne    gm    0.636801        3
7       tsne    km    0.636801        3
56       svd    gm    0.625019        3
28       pca    km    0.625019        3
35       pca    gm    0.625019        3
49       svd    km    0.625019        3
2       tsne   agg    0.608399        5
21       pca   agg    0.601363        3
42       svd   agg    0.601363        3
22       pca   agg    0.571247        4
43       svd   agg    0.571247        4


In [99]:
# получаем метки кластеров с помощью моделей, показавших наибольшее значение коэффициета силуэта
# для трех, четырех и пяти кластеров
gm_3 = GaussianMixture(n_components=3, random_state=42)
country_data['cluster_3'] = gm_3.fit_predict(res_country[1]['svd'])

agg_4 = AgglomerativeClustering(4, linkage='average', metric='l2')
agg_4.fit(res_country[1]['pca'])
country_data['cluster_4'] = agg_4.labels_

agg_5 = AgglomerativeClustering(5, linkage='average', metric='l2')
agg_5.fit(res_country[1]['tsne'])
country_data['cluster_5'] = agg_5.labels_

country_data.head() # итоговый датасет с тремя вариантами кластеризации стран

,region,language,country_dist,uniq_custID_ratio,population,cluster_3,cluster_4,cluster_5
Country,,,,,,,,
United Kingdom,Northern Europe,en,0,90.459690,64105654,2,1,0
Netherlands,Western Europe,nl,358,0.207900,16881000,2,1,4
Germany,Western Europe,de,933,2.171402,80783000,2,1,0
France,Western Europe,fr,344,2.009702,66078000,2,1,0
Switzerland,Western Europe,de,749,0.485100,8183800,2,1,3


In [100]:
# создаем словари для всех вариантов кластеризации (для 3, 4 и 5 кластеров)

country_3_clust_dict = dict()
for i  in range(3):
    country_3_clust_dict[i] = list(country_data[country_data['cluster_3'] == i].index)

country_4_clust_dict = dict()
for i  in range(4):
    country_4_clust_dict[i] = list(country_data[country_data['cluster_4'] == i].index)

country_5_clust_dict = dict()
for i  in range(5):
    country_5_clust_dict[i] = list(country_data[country_data['cluster_5'] == i].index)    

Посмотрим на получившиеся кластеры стран

In [101]:
country_3_clust_ser = pd.Series(country_3_clust_dict)
country_4_clust_ser = pd.Series(country_4_clust_dict)
country_5_clust_ser = pd.Series(country_5_clust_dict)
print('Состав кластеров при разделении на три кластера:')
print(country_3_clust_ser)
print()
print('Состав кластеров при разделении на четыре кластера:')
print(country_4_clust_ser)
print()
print('Состав кластеров при разделении на пять кластеров:')
print(country_5_clust_ser)

Состав кластеров при разделении на три кластера:
0              [Japan, Australia, Canada, Brazil, USA]
1    [Lebanon, United Arab Emirates, Israel, Saudi ...
2    [United Kingdom, Netherlands, Germany, France,...
dtype: object

Состав кластеров при разделении на четыре кластера:
0                 [Japan, Australia, Canada, USA, RSA]
1    [United Kingdom, Netherlands, Germany, France,...
2    [Lebanon, United Arab Emirates, Israel, Saudi ...
3                                             [Brazil]
dtype: object

Состав кластеров при разделении на пять кластеров:
0    [United Kingdom, Germany, France, Portugal, It...
1                                    [Poland, Iceland]
2                                     [Czech Republic]
3                                        [Switzerland]
4                                        [Netherlands]
dtype: object


Разделение стран на три кластера лучше всего поддается интерпретации. В дальнейшей работе будем использовать именно его

In [102]:
def get_country_label(x, country_clust_dict, n):
    
    """Функция возвращает метку кластера по названию страны
       
    Args:
        x (str): страна клиента
        country_clust_dict (dict): словарь с парами кластер-список стран
        n (int): число кластеров

    Returns:
        _type_: _description_
    """
    for i in range(n):
        if x in country_clust_dict[i]:
            return i
        
# создаем признак принадлежности страны клиента к определенному кластеру
data_cleaned['country_cluster_3'] = data_cleaned['Country'].apply(lambda x: get_country_label(x, country_3_clust_dict, n=3))   
# data_cleaned['country_cluster_4'] = data_cleaned['Country'].apply(lambda x: get_country_label(x, country_4_clust_dict, n=4))  
# data_cleaned['country_cluster_5'] = data_cleaned['Country'].apply(lambda x: get_country_label(x, country_5_clust_dict, n=5))       

Теперь сгруппирем данные по клиентам.

In [103]:
def get_group_rfm_country(data_for_group, feature):
    """функция группирует данные по клиентам, создавая rfm-признаки, 
    а также признак, указывающий на кластер страны клиента

    Args:
        data_for_group (dataframe): данные для группировки
        feature (str): название дополнительного признака

    Returns:
        dataframe: сгруппированные по клиентам данные
    """
    group_data = data_for_group.groupby(by='CustomerID')[['InvoiceDate', 
                                                             'InvoiceNo', 
                                                             'TotalPrice', 
                                                             feature]].agg({'InvoiceDate' : 'max',
                                                                                       'InvoiceNo' : 'nunique',
                                                                                       'TotalPrice' : 'sum',
                                                                                       feature : pd.Series.mode})
                                                        
    group_data['Recency'] = (day_d - group_data['InvoiceDate']).dt.days  
    group_data.rename(columns={'InvoiceNo' : 'Frequency', 
                                    'TotalPrice' : 'Monetary'}, inplace=True) 
    group_data.drop('InvoiceDate', axis=1, inplace=True) 
    
    return group_data



rfm_country_3_table = get_group_rfm_country(data_cleaned, 'country_cluster_3')

# подготовим данные для обучения
ct_3 = make_column_transformer((OneHotEncoder(), ['country_cluster_3']),
                             (MinMaxScaler(), ['Frequency', 'Monetary', 'Recency']))
enc_rfm_country_3_table = ct_3.fit_transform(rfm_country_3_table)

Подбираем способ кластеризации клиентов 

In [104]:
res_sil_3_country_cluster = get_silhouette_res_and_scatter(enc_array=enc_rfm_country_3_table, 
                                                           index_data=rfm_country_3_table, 
                                                           scaler=StandardScaler(), 
                                                           p=15, 
                                                           n=3)
print('Резултаты подбора вариантов кластеризации клиентов при разделении стран на три кластера')
print(res_sil_3_country_cluster[0].head(15))

Резултаты подбора вариантов кластеризации клиентов при разделении стран на три кластера
   reduction model  silhouette  cluster
21       pca   agg    0.635878        3
42       svd   agg    0.635878        3
43       svd   agg    0.633344        4
22       pca   agg    0.633344        4
23       pca   agg    0.592360        5
44       svd   agg    0.592360        5
52       svd    km    0.559724        6
31       pca    km    0.559724        6
51       svd    km    0.555832        5
30       pca    km    0.555832        5
32       pca    km    0.543218        7
53       svd    km    0.543218        7
48       svd   agg    0.542514        9
27       pca   agg    0.542514        9
35       pca    gm    0.541391        3


Добавим к rfm-признакам приизнак принадлежности страны клиента к определенному кластеру мы смогли повысить значение коэффициента силуэта кластеров клиентов.

Получаем кластеры клиентов с помощью модели показавшей наилучший результат

In [105]:
agg_3 = AgglomerativeClustering(3, linkage='average', metric='l2')
agg_3.fit(res_sil_3_country_cluster[1]['pca'])
rfm_country_3_table['cluster_3'] = agg_3.labels_
print('Количество клиентов в кластерах: ')
rfm_country_3_table['cluster_3'].value_counts()

Количество клиентов в кластерах: 


cluster_3
1    3958
0      31
2      31
Name: count, dtype: int64

Так как размерность  rfm_country_3_table больше трех, мы не можем создать интерпретируемую диаграмму рассеяния. Однако нам доступна визуализация профилей полученных кластеров с помощью полярных диаграмм.

In [106]:
group_rfm_country = rfm_country_3_table.groupby(by='cluster_3')[['Frequency', 
                                                                 'Monetary', 
                                                                 'Recency',  
                                                                 'country_cluster_3']].mean() 
print(group_rfm_country)
plot_cluster_profile(group_rfm_country, 3, m=2)

           Frequency     Monetary     Recency  country_cluster_3
cluster_3                                                       
0           1.806452  1067.615806  156.935484           0.290323
1           2.908540   929.842244   98.028297           2.000000
2          11.161290  4808.344516   16.677419           2.000000


Охарактеризуем полученные кластеры:
* кластер 0: малочисленный кластер (31 клиент), преимущественно состоящий из клиентов из дальних стран (Япония, Бразилия, США, Автстралия, Канада) и стран ближневосточного региона,  редкие покупки, малая выручка, максимальная давность совершения последней покупки, "в зоне потери"
* кластер 1: самый многочисленный кластер (3958 клиентов) европейцев, малая выручка и частота заказов, средняя давность последней покупки, "сомневающиеся" или "спящие"
* кластер 2: малочисленный кластер (31 клиент) постоянных клиентов-европейцев, с максимальными показателями выручки и частоты покупок, срок давности последней покупки минимален, "лояльные"

Теперь попробуем повлиять на кластеризацию клиентов, добавив информацию о времени оформления заказа 

In [107]:
data_time = data_cleaned[['week_day', 'hour', 'month', 'quarter', 'CustomerID' ]]
data_time.reset_index(drop=True, inplace=True)

# группируем временные характеристики по клиентам, находя модальные значения
group_time = data_cleaned.groupby(by='CustomerID')[['week_day', 'hour', 'month', 'quarter']].agg({'week_day' : pd.Series.mode,
                                                                                                  'hour' : pd.Series.mode,
                                                                                                  'month' : pd.Series.mode,
                                                                                                  'quarter': pd.Series.mode})
# обрабатываем случаи, когда модальное значение не одно
group_time['hour'] = group_time['hour'].apply(lambda x: x if type(x) == np.int32 else x[0])
group_time['week_day'] = group_time['week_day'].apply(lambda x: x if type(x) == np.int32 else x[0])
group_time['month'] = group_time['month'].apply(lambda x: x if type(x) == np.int32 else x[0])
group_time['quarter'] = group_time['quarter'].apply(lambda x: x if type(x) == np.int32 else x[0])

# кодируем признаки, стандартизация/нормализация не требуется
group_time_enc = pd.get_dummies(group_time, columns=['week_day', 'hour', 'month', 'quarter'])
# приступаем к подбору 
res_time = get_silhouette_res_and_scatter(scaler=RobustScaler(), enc_array=group_time_enc, index_data=group_time, p=15, n=4)
print("Результаты подбора метода кластеризации параметров времени оформления заказа:")
print(res_time[0])

Результаты подбора метода кластеризации параметров времени оформления заказа:
   reduction model  silhouette  cluster
21       pca   agg    0.893429        3
28       pca    km    0.893429        3
35       pca    gm    0.889268        3
20      tsne    gm    0.851342        9
6       tsne   agg    0.851342        9
..       ...   ...         ...      ...
41       pca    gm    0.437261        9
60       svd    gm    0.429258        7
53       svd    km    0.419116        7
40       pca    gm    0.415656        8
62       svd    gm    0.388983        9

[63 rows x 4 columns]


In [108]:
# получаем метки кластера времени заказа для каждого клиента
km = KMeans(n_clusters=3, random_state=42, init='random')
km.fit(res_time[1]['pca'])
group_time['cluster_time'] = km.labels_
print('Количество клиентов в кластерах:')
group_time['cluster_time'].value_counts()

Количество клиентов в кластерах:


cluster_time
2    1640
1    1476
0     904
Name: count, dtype: int64

In [109]:
# посмотрим на усредненные параметры кластеров времени оформления заказа
group_time.groupby(by='cluster_time')[['week_day', 'hour', 'month','quarter']].agg({'week_day' : pd.Series.mode,
                                                                                                  'hour' : pd.Series.mode,
                                                                                                  'month' : pd.Series.mode,
                                                                                                  'quarter': pd.Series.mode})

,week_day,hour,month,quarter
cluster_time,,,,
0,3,12,9,3
1,3,12,3,2
2,3,12,11,4


Как мы видим, кластеры отличаются лишь самым популярным  месяцем и, соответственно, кварталом года. Возможно это связано с праздничными датами в Великобритании

In [110]:
# удаляем результаты предыдущей кластеризации клиентов
rfm_country_time_table = rfm_country_3_table.drop('cluster_3', axis=1)
# добавляем в сгруппированные по клиентам данные метки кластеров времени
rfm_country_time_df =  pd.merge(rfm_country_time_table, 
                                group_time, 
                                left_index=True, 
                                right_index=True)
# удаляем ненужные признаки
rfm_country_time_df.drop(['week_day', 'hour',	'month', 'quarter'], axis=1, inplace=True)
rfm_country_time_df.head()

,Frequency,Monetary,country_cluster_3,Recency,cluster_time
CustomerID,,,,,
12346,1,0.00,2,325,1
12347,7,4310.00,2,2,2
12348,4,1437.24,2,75,2
12349,1,1457.55,2,18,2
12350,1,294.40,2,310,1


In [111]:
# подготовим данные для обучения
ct = make_column_transformer((OneHotEncoder(), ['country_cluster_3', 'cluster_time']),
                             (MinMaxScaler(), ['Frequency', 'Monetary', 'Recency']))

enc_rfm_country_time = ct.fit_transform(rfm_country_time_df)

In [112]:
# подбираем лучший метод кластеризации
res_rfm_country_time = get_silhouette_res_and_scatter(scaler=StandardScaler(), 
                                                      enc_array=enc_rfm_country_time, 
                                                      index_data=rfm_country_time_df, 
                                                      p=15, n=5)[0]
print('Результаты подбора метода кластеризации клиентов:')
print(res_rfm_country_time.head(15))

Результаты подбора метода кластеризации клиентов:
   reduction model  silhouette  cluster
29       pca    km    0.583925        4
50       svd    km    0.583925        4
22       pca   agg    0.563581        4
43       svd   agg    0.563581        4
23       pca   agg    0.561315        5
44       svd   agg    0.561315        5
24       pca   agg    0.561091        6
45       svd   agg    0.561091        6
31       pca    km    0.550331        6
52       svd    km    0.550331        6
5       tsne   agg    0.548171        8
51       svd    km    0.547394        5
30       pca    km    0.547394        5
25       pca   agg    0.546088        7
46       svd   agg    0.546088        7


Использование информации о времени совершения заказа не принесло ожидаемого результата. Самым выгодным с точки зрения величины коэффициента силуэта оказался вариант, представленный в rfm_country_3_table. 

Так же одним из возможных способов повышения качества кластеризации является выявление категории купленного товара из его описания

## 5. Выводы и оформление работы

Напишите заключение о проделанной работе и полученных результатах. Отметьте, какие сегменты клиентов вам удалось найти в данных.

После завершения проекта создайте в своём репозитории файл README.md и кратко опишите содержимое проекта по принципу, который мы приводили ранее.

Выложите свой проект на GitHub и оформите удалённый репозиторий, добавив в него описание и теги репозитория (теги придумайте самостоятельно в зависимости от того, какую задачу вы решали).

Применительно к исходному датасету наиболее выигрышно показали себя сингулярное разложение и метод главных компонент. Данные пониженной размерности, полученные этими методами, во многом схожи. Кластеры полученные на основе этих данных лучше поддаются интерпретации. Результаты кластеризации сильно зависят, как от параметров понижения размерности, так и от параметров методов кластеризации.  Однако при принятии решения о применениии того или иного метода кластеризации следует опираться помимо мер качества на здравый смысл и задачи,поставленные маркетологами.